In [52]:
path = r'C:\Users\97250\Desktop\studied\R ,python\Datasets Kaggle\smallData_globalterrorism.csv'

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import mean_absolute_error,mean_squared_error, accuracy_score ,recall_score ,precision_score

import numbers

In [54]:
data = pd.read_csv(path)

In [55]:
def read():
    path = r'C:\Users\97250\Desktop\studied\R ,python\Datasets Kaggle\smallData_globalterrorism.csv'
    data = pd.read_csv(path)
    return data

In [7]:
data.columns

Index(['Unnamed: 0', 'id', 'year', 'month', 'dayInMonth', 'extended?',
       'country', 'region', 'provstate', 'success', 'suicide', 'attacktype1',
       'targtype1', 'targsubtype1', 'corp1', 'gname', 'motive', 'individual',
       'claimed?', 'claimmode', 'weaptype1', 'weapsubtype1_txt', 'weaptype2',
       'weapsubtype2', 'weapdetail', 'nkill', 'nwound', 'property?',
       'propertyDemaged', 'ishostkid', 'ransom', 'ransomamt', 'nreleased',
       'dbsource', 'sumSites'],
      dtype='object')

In [ ]:
outcome = ['','','','']  #  Needed data
color = ['black','red','green','gold']

In [7]:
def findRelevantDataColumns():
    
    """
    This function seperates columns into relvance levels.
    
    Input  - None
    Output - 
    """
    time = []
    rel  = []
    unrel= []
    cont = []
    
    for i in data.columns:
        
        if type(data[i][0]) == str: # if the value is stringy
            
            #time columns
            if ''.join(i).lower() in "month dat dayinmonth dayinweek year date exactdate".split() :
                time.append(i)
            # small values spread
            elif len(data[i].value_counts()) <30:
                rel.append(i)
            # too many categories
            else:
                unrel.append(i)
        
        else: #if values are numerical or else(bool[binary too] , list(?))
            cont.append(i)
    
    if len(data.columns) < 25:
        print(time,rel,unrel,cont,sep = '\n')
    
    return time,rel,unrel,cont

In [6]:
def show(data,kind ='bar',size = 'm',title='',save=False):
    
    """
    This is a visualization automatic function.
    
    
    Parameters - None of these must be inserted.
    ----------
    
    data - Already given.
        DESCRIPTION: The data to plot visualization for.
            
    kind : string. Default: 'bar'.
        DESCRIPTION: plot kind. can be: bar, line,time,heat,stacked.
  
    size : string. Default: 'm' (medium size).
        DESCRIPTION: size of plot.

    title: string. Default:''.
        DESCRIPTION: title string.

    save: bool. Default:False.
        DESCRIPTION: Wether to save output or not.


    Returns
    -------
    TYPE: NoneType.
        DESCRIPTION: Visualization for given data. 
     
    """
    
    dic = {'l':(16,12),'m':(12,8),'s':(8,6)}    
    scale = dic[size]
    
    if kind=='bar':
        #if num categories is 1:
        #else: stacked bar
        data.plot(kind = 'bar',color = 'gold',figsize = scale,fontsize = scale[0]+4)
    elif kind=='line':
        data.plot(kind = 'line', lw = (scale[0]//4),figsize = scale,fontsize = scale[0]+4)
    elif kind=='time':
        data.sort_index().plot(kind = 'line',color = color[0], lw = (scale[0]//4),figsize = scale,fontsize = scale[0]+4)
    elif kind=='heat':
        plt.figure(figsize = scale)
        sns.heatmap(data = data ,lw=1 ,linecolor = 'white',cmap = 'Reds', annot = True)
    elif kind == 'stacked':
        data.plot(kind = 'bar', stacked=True,figsize = scale,fontsize = scale[0]+4)
        
   
    ### Do anyway  ###
    
    sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1.8, color_codes=True, rc=None)
    plt.title("\n " + title + " \n",fontsize = 2*scale[0])
    #Information box
    plt.legend(loc='upper right',prop={'size':1.2*scale[0]}) 
    
    # Save?
    
    if save:
        
        path = r'C:\Users\97250\Desktop\studied\R ,python\Datasets Kaggle\Outputs data\globalTerrorism\''
        file = path.split('\'')[-2]
        
        ###  verifying current location
        x = input('Your current file name is'+ file+'\n are you sure you want to save plots there? ')
        if x.lower() not in ['y','yes']:
            newFile = input('Please write the directory you want it to be saved in: \n')
            path.replace(file,newFile)
        
        # Saving
        plt.savefig(path + title + '.png')


In [36]:
#most common publishers

def toppers(agg,colum,outcome,top=10):
    
    """
    This func return data for top parameter.
    """
    #toppers
    toppers = agg[outcome].sort_values(ascending=False)[:top].index
    
    #grab aggregated data that is in these toppers.
    agg = agg.reset_index()
    dataOfToppers = agg[agg[colum].isin(toppers)]    
    
    # just some tecnique in order to present the name of toppers in the visual graph later.
    dataOfToppers = dataOfToppers[[colum,outcome]].set_index(colum).sort_values(ascending = False,by = outcome)
    
    return dataOfToppers

In [7]:
def agg(colum ,k='sum'):
    
    """
    Aggregation function.
    """
    
    agg = data.groupby(colum)
   
    # These function change due to the data kind I work with.
    agg = agg.sum() if k=='sum' else agg.mean() if k=='mean' else agg.count() if k=='count' else agg.var()
    
    return agg

In [10]:
def aggPlot(colum , i =0 ,k ='sum',size = 'm', title='' ,top=10,save=False,data=data):
    
    """
    Aggregation and plot function.
    
    Input:
        Colum: string or list of strings.- columns to aggregate with.
        i: int, kind of column to agg' on from list 'outcome'.
        k: string,  sum/mean else-count.
        size: char['s','m','l']  -scale of output
        title: string -title to print
        save: bool - wether to save picture
        
    Return:
        DataFrame's aggregated column(c).
    """
    
    c = outcome[i]
    title =  k.capitalize() +' of '+ title
    
    #calling the agg func.
    agg = agg(colum=colum,k=k)
    
    
    ###     Visualisation     ###

    #If time line elements exist:
    for i in colum.lower():
        if i in ['year','date']:
            return show(data = agg[c] ,kind ='time' ,size = size, title = title)
    
    else:  # No TimeLine elements:
        
    # if tow columns given - most likely this accurs:
        if len(agg[c].value_counts()) > 50:  # too many categories
            agg = toppers(agg = agg ,colum=colum , top=top ,outcome = c)
            return show(data = agg  ,kind ='bar' ,size = size, title=title+' per '+str(top)+' top '+colum )

        elif type(colum) != list: # if one column has been given:
            agg = agg[c].sort_values(ascending=False)
            return show(data = agg ,kind ='bar' ,size = size, title= title)

        else: # tow dimentional data or more.
            x = input('Sir, to unstack?  (y/n)')

            if x == 'y':
                agg =agg[c].unstack(0)
                return show(data=agg,kind='heat',size = size, title = title )

            elif x =='n': 
                agg = toppers(agg = agg, colum = colum, outcome = c, top=top)
                return show(data=agg,kind='bar',size = size, title = title+' per '+str(top)+' top '+colum) #perhaps a line....
                                    #or stacked bar...

                ## additional option:dataframe: agg[[c]].sort_values(ascending = False,by = c


NameError: name 'data' is not defined

In [11]:
def aggPlot(colum , i =0 ,k ='sum',size = 'm', title='' ,top=10,save=False,data=data):
    
    """
    Aggregation annd plot function.
    
    Input:
        Colum: string or list of strings.- columns to aggregate with.
        i: int, kind of column to agg' on from list 'outcome'.
        k: string,  sum/mean else-count.
        size: char['s','m','l']  -scale of output
        title: string -title to print
        save: bool - wether to save picture
        
    Return:
        DataFrame's aggregated column(c).
    """
    
    c = outcome[i]
    title =  k.capitalize() +' of '+ title
    timeExpresion = ['year','date']
    
    agg = agg(colum=colum,k=k,i=i)
    
    #visuallize 
    
    if type(colum) == list:
        
        if len(colum)==2:

            for i in colum.lower():
                if i in timeExpresion:       
                    return show(data = agg[c] ,kind ='time' ,size = size, title = title)

            #else: ## missing line'stacked bar .
            agg = agg[c].unstack(0)
            return show(data=agg,kind='heat',size = size, title = title )
        
    
        else: #more than 2:
            #ordinary form
            agg[[c]].sort_values(ascending = False,by = c)
    
    else:    #one agg column
        
        #Time line
        if colum.lower() in timeExpresion:       
            return show(data = agg[c] ,kind ='time' ,size = size, title = title)
        
        #bar
        else: 
            return show(data = agg[c] ,kind='bar',size = size, title = title )

NameError: name 'data' is not defined

In [2]:
###   This method is new and unedited

In [3]:
def run():
    
    """
    Ment to find relevant columns
    
    Param: None
    Output: graphs
    
    """
    
    print('')

In [87]:

def label():
    
    labelColum = input("Dear user, please insert the column name of the label: ")

    # Error preventing
    while labelColum not in data.columns:
        print('The optins are: ', data.columns)
        labelColum = input("Dear user, please insert the column name of the label. Properly this time...: ")

    print('\n\nThe column chosen as label is :' ,labelColum)

    if isinstance(data[labelColum][0] , numbers.Number):  # needed to write more    
        labelKind_isContinuous = True

    else:  #binary
        labelKind_isContinuous = False

        # multi class error
        if len(data[labelColum].value_counts())>2:
            print("This might be a multiclass problem. \n Sorry but I can't  help here.")
            print("The system has stopped since multiclass-task has forced upon it.")


In [23]:
def highStd(colum):
    
    """
    
    Checks if the data is highly spread(first tow are more than 80% of the data)
    Usefull for binary colum check since there are always some UNknown values and other....
    
    """
    top,low = 0,0

    for c,i in enumerate(data[colum].value_counts()):
        if c<2:# Tow firsts big value_counts.
            top+=i
        else:
            low+=i

    if top/5 > low: # much more tow firsts than the rest(5 times over). 
        return True # high Std.
    
    return False    #low Std.
    

In [24]:
def binary(colum):
    
    if len(data[colum].value_counts())<=2: ##4 and highStd(colum):
        return True
    
    return False

In [25]:
def checker(colum):
    
    for i in range(10):
        
        if type(data[colum][i]) != type(data[colum][i+1]):
            return False
    
    return True
        

In [26]:

import numbers


def ColumnSegmentation():

    
    """
    Column Segmentation.
    
    """
    smallCategoryColumns = []
    largeCategoryColumns = []
    binaryColumns  = []
    continuousColumns = []

    
    for i in data.drop(labelColum,axis=1).columns: # without the label
        
        if checker(i):

            x = type(data[i][0]) # first value of column   

            if binary(i) :
                binaryColumns.append(i)

            elif isinstance(x , numbers.Number) and not binary(i):
                continuousColumns.append(i)

            elif x == str:

                if len(data[i].value_counts())<20:
                    smallCategoryColumns.append(i)

                elif len(data[i].value_counts())<100:
                    largeCategoryColumns.append(i)
           
        else: 
            print('ununified form of data has been found at the column :   ' + i)
            
                
    return binaryColumns, continuousColumns, smallCategoryColumns,largeCategoryColumns

In [44]:

def categorialDummy(colum):
   
    global data
    """
    Seperates categorial columns into dummy features.
     
    Input : The column to make dummy variables on.
    Output : 'Dummied' data. 
    
    """
    
    dummies = pd.get_dummies(data[colum],drop_first=True)
    data = pd.concat([data.drop(colum,axis=1),dummies], axis=1)
    

    

In [56]:
data = read()

In [59]:
data.head()

,Unnamed: 0,id,year,month,dayInMonth,extended?,region,provstate,success,suicide,...,Uzbekistan,Venezuela,Wallis and Futuna,West Bank and Gaza Strip,West Germany (FRG),Yemen,Yugoslavia,Zaire,Zambia,Zimbabwe
0,14878,1.980000e+11,1981,11,27,0,Central America & Caribbean,Francisco Morazan,1,0,...,0,0,0,0,0,0,0,0,0,0
1,84463,2.010000e+11,2007,10,15,0,South Asia,Southern,1,0,...,0,0,0,0,0,0,0,0,0,0
2,181370,2.020000e+11,2017,12,16,0,South Asia,Khulna,1,0,...,0,0,0,0,0,0,0,0,0,0
3,28000,1.990000e+11,1986,5,8,0,South America,Bogota,1,0,...,0,0,0,0,0,0,0,0,0,0
4,58070,1.990000e+11,1994,12,20,0,East Asia,Tokyo,1,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
categorialDummy('country')

In [48]:
data

,Unnamed: 0,id,year,month,dayInMonth,extended?,region,provstate,success,suicide,...,Uzbekistan,Venezuela,Wallis and Futuna,West Bank and Gaza Strip,West Germany (FRG),Yemen,Yugoslavia,Zaire,Zambia,Zimbabwe
0,14878,1.980000e+11,1981,11,27,0,Central America & Caribbean,Francisco Morazan,1,0,...,0,0,0,0,0,0,0,0,0,0
1,84463,2.010000e+11,2007,10,15,0,South Asia,Southern,1,0,...,0,0,0,0,0,0,0,0,0,0
2,181370,2.020000e+11,2017,12,16,0,South Asia,Khulna,1,0,...,0,0,0,0,0,0,0,0,0,0
3,28000,1.990000e+11,1986,5,8,0,South America,Bogota,1,0,...,0,0,0,0,0,0,0,0,0,0
4,58070,1.990000e+11,1994,12,20,0,East Asia,Tokyo,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9079,12371,1.980000e+11,1980,11,22,0,Middle East & North Africa,Gaza Strip,1,0,...,0,0,0,1,0,0,0,0,0,0
9080,67134,2.000000e+11,1997,11,17,0,Sub-Saharan Africa,Ruhengeri,1,0,...,0,0,0,0,0,0,0,0,0,0
9081,56317,1.990000e+11,1994,5,17,0,South America,Lima,1,0,...,0,0,0,0,0,0,0,0,0,0
9082,15709,1.980000e+11,1982,3,16,0,Middle East & North Africa,Beirut,1,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
def nullConvertor():
    
    """
    Convert all X data Types to None:

    """
    
    x = input("Dear user!\n This is a critical point at analyzing this data. \n Please insert the way null represented here.")
    
    for i in data.columns:
        data[i].replace(x,None)  #surely there is more efficient way
        

In [29]:
def binaryConvertor():
    
    """
    United format of binary columns.
    """
    
    def replaces(x):
        #return x.replace(dict.fromkeys([True,'YES','Y','yes'],1).replace(dict.fromkeys([False,'no','N','NO'],0))
        return x.replace(True,1).replace(False,0).replace('yes',1).replace('no',0).replace('YES',1).replace('NO',0)

    
    for i in binaryColumns:
        data[i] = data[i].apply(replaces)


In [30]:
def nullColumDropper(thresh=0.4):
    
    """
    Drop columns with too many null values.
    
    """
    length = len(data)
    
    for i in data.columns:  
        
        ratio = data[i].isnull().sum() / length
        
        if ratio > thresh:  #40% missing data
            data.drop(i,axis=1,inplace=True)
            print('The column '+i+' has been dropped. ' + str(round(ratio*100,2)) + '% null values. ')
            


In [31]:
def nullRowsDropper(data):
    
    x = data.isnull().sum()
    
    threshold = x.sum() / len(x)
    
    columnsToDrop = x[x>threshold].index
    
    data.drop(columnsToDrop ,axis=1 ,inplace = True)
    
    print('\n\nThe following columns were dropped for having relatively many missing values:\n\n' ,(round(x[x>threshold]/len(data)*100).sort_values(ascending =False)))
    
    return data

In [32]:
def countUniqueNullRows(data):
    
    """
    count unique null rows.
    """
    
    sumNullRows = 0
    
    
    for index, row in data.iterrows():
        if row.isnull().sum()>0:
            sumNullRows += 1
            
    return sumNullRows

In [33]:
def nullHandling(thresh=0.4):
    
    """
    Null handling.
    """
    
    global data
    
    #Drop columns with too many null values.   
    nullColumDropper(thresh = thresh)

    #Drop rows if low total null
    rowsWithNullValues = countUniqueNullRows(data)
    
    while rowsWithNullValues > 0.1*len(data):
        data = nullRowsDropper(data)
        rowsWithNullValues = countUniqueNullRows(data)
            
    ratioNullRows = str(round(rowsWithNullValues/len(data)*100,2))
    print('\n\n The current amount of records with null values now is ' +str(rowsWithNullValues)+', which is '+ ratioNullRows+'% of the total records.')
    print('Now I will delete only these rows')
    print('The shape of the data now is '+str(data.shape)+' instead of ')
    
    data.dropna(how='any',axis=0,inplace=True)


In [34]:
path = r'C:\Users\97250\Desktop\studied\R ,python\Datasets Kaggle\smallData_globalterrorism.csv'
data = pd.read_csv(path)

In [35]:
binaryColumns, continuousColumns, smallCategoryColumns,largeCategoryColumns = ColumnSegmentation()

ununified form of data has been found at the column :   targsubtype1
ununified form of data has been found at the column :   corp1
ununified form of data has been found at the column :   motive
ununified form of data has been found at the column :   weapdetail


In [60]:
def main():
    
    """
    Father function of choosing columns.
    """
    
   # for i in data.columns:
   #     data[i] = data[i].apply(lambda x : str(x).lower().replace('unknown',None))
    
    #print('I have switched \'unkonown\' to null and treat them as missing values.')
    
    nullHandling()
     
    label()
    
    binaryColumns, continuousColumns, smallCategoryColumns,largeCategoryColumns = ColumnSegmentation()
    
    for i in data.columns:
        
        # null connect method
        
        if i in smallCategoryColumns or i in largeCategoryColumns:
            categorialDummy(i)
        
        elif i in binaryColumns:
            binaryConvertor(i)

In [ ]:
# Relative evaluation function

def algoIsSutisfying(y_test, predictions):
    
    if labelKind_isContinuous:
        
        labelAvg =data[labelColum].mean()
        MAE = round(mean_absolute_error(predictions,y_test),5)
        rationalMAE = round(MAE/labelAvg,3)
        
        if rationalMAE < 3 * labelAvg :
            return True
        
    else:
         if accuracy_score(y_test, predictions)>0.8:
            return True
        
    return False

In [ ]:
def predict(algo):
    
    kind = ['treeRegressor','treeClassifier','randomForestRegressor','randomForestClassifier','randomForestRegressor450','randomForestClassifier450'][algo]
    kind = kind[algo]
    
    if kind == 'treeRegressor' :
        algorithm = DecisionTreeRegressor()
        
    elif kind == 'treeClassifier' :
        algorithm = DecisionTreeClassifier()
        
    elif kind == 'randomForestRegressor' :
        algorithm = RandomForestRegressor(n_estimators = 150)
        
    elif kind == 'randomForestClassifier' :
        algorithm = RandomForestClassifier(n_estimators = 150)
    
    elif kind == 'randomForestRegressor450' :
        algorithm = RandomForestRegressor(n_estimators = 450)
        
    elif kind == 'randomForestClassifier450' :
        algorithm = RandomForestClassifier(n_estimators = 450)        
   

    algorithm.fit(X_train,y_train)

    predictions = algorithm.predict(X_test)
    
    
    while ! algoIsSutisfying(y_test, predictions):
        or loop<3 and dataDimention<2*10**6:
        predict(algo+2) #meaning the randomForest version.
    
    return algorithm values
    

In [ ]:



def calculate():
   

    # train/test splitting
    
    X = final_data.drop(labelColum,axis=1)
    y = final_data[labelColum]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

    
    if labelKind_isContinuous:
        predict(algo = 0)
    
    else:
        predict(algo = 1)
    

In [1]:
# extracting timeData

def timeExtraction():

    for i in data.columns:
        if i.lower() in ['year','date','datetime','time'] :
            global timeColum
            timeColum = i # assuming only one colum

    if len(str(data[timeColum][0]))>4: # not only year value
    
        data[timeColum] = data['date'].apply(lambda x : pd.to_datetime(x))

        data['month'] = data[timeColum].apply(lambda x : x.month)
        data['day of month'] = data[timeColum].apply(lambda x : x.day)

        dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
        data['day of week'] = data[timeColum].apply(lambda x:dmap[x.dayofweek])


In [81]:
global x
def r():
    
    x=5
    

In [82]:
r()

In [83]:
x

5